## Problem 5

In [ ]:
## For this question, fill in the TODO's 

In [ ]:
!pip install git+https://github.com/adapter-hub/adapter-transformers.git
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks RTE

import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModelWithHeads, AdapterType, AutoConfig, AutoModelForSequenceClassification
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

model_name = 'bert-base-uncased'


# Refer to the notebook for training an adapter to write these. Set the number of epochs to 3, and learning rate to 5e-5. Rest of the hyperparameters can stay the same. 

data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./glue_data/RTE")

training_args = TrainingArguments(
    logging_steps=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    save_steps=1000,
    evaluation_strategy='steps',
    output_dir='./models/rte',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    learning_rate=5e-5,
    num_train_epochs=3,
)

# TODO: Change this seed when re-running your code to report the mean and std dev
#set_seed(2020)
#set_seed(666)
set_seed(1024)
num_labels = glue_tasks_num_labels[data_args.task_name]


  Cloning https://github.com/adapter-hub/adapter-transformers.git to /tmp/pip-req-build-nyf7a393
  Running command git clone -q https://github.com/adapter-hub/adapter-transformers.git /tmp/pip-req-build-nyf7a393
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for adapter-transformers: filename=adapter_transformers-1.1.0-cp36-none-any.whl size=1325191 sha256=6dcc5d87a50a188bd9521fe52d4401f2dfb047816ad57d142df2691194651bda
  Stored in directory: /tmp/pip-ephem-wheel-cache-i3vmfd4e/wheels/22/eb/df/1c86c6a1b0323a74470d6a53db05b3b49ec79bce18d253ec38
Successfully built adapter-transformers
fatal: destination path 'transformers' already exists and is not an empty directory.
	Completed!


In [ ]:
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=".",
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=".",
)

model = AutoModelWithHeads.from_pretrained(model_name, config=config)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now we have everything set up to load our AdapterFusion setup. 

First, you need to go to adapterhub.ml and explore the different adapters available. Choose any three adapters and load the three adapters. As we don't need their prediction heads, we pass with_head=False to the loading method. Next, we add a new fusion layer that combines all the adapters we've just loaded. Finally, we add a new classification head for our target task on top.

In [ ]:
# First, load the pre-trained adapters we want to fuse from Hub
from transformers.adapter_config import PfeifferConfig

model.load_adapter("nli/rte@ukp", "text_task", config=PfeifferConfig(), load_as='rte', with_head=False)
# TODO: load some more adapters. Choose the ones that you think will help RTE task after reading about the different tasks available and how big the datasets are.
model.load_adapter('sts/mrpc@ukp', config=PfeifferConfig(), load_as='mrpc', with_head=False)
model.load_adapter("nli/multinli@ukp", "text_task", config=PfeifferConfig(), load_as='mnli', with_head=False)
model.load_adapter("nli/sick@ukp", "text_task", config=PfeifferConfig(), load_as='sick', with_head=False)



# Add a fusion layer and tell the model to train fusion (freezes the rest of the weights) (here can either add the actual atsk adapter or not)
model.add_fusion([
        "rte",
        # TODO: Add your other task names here for the adapters you chose
        'mrpc',
        'mnli',
        'sick'
    ])

# Add a classification head for our target task
# TODO: Check the earlier notebook from Problem 5 to see how to add a classification head for your task.
model.add_classification_head('rte', num_labels=num_labels)

The last preparation step is to define and activate our adapter setup. Similar to train_adapter(), train_fusion() does two things: It freezes all weights of the model (including adapters!) except for the fusion layer and classification head. It also activates the given adapter setup to be used in very forward pass.

The syntax for the adapter setup (which is also applied to other methods such as set_active_adapters()) works as follows:

a single string is interpreted as a single adapter, 
a list of strings is interpreted as a stack of adapters,
a nested list of strings is interpreted as a fusion of adapters. Here want to do Fusion so we use a nested list as follows. 


In [ ]:
# QUESTION: What's the difference between a stack of adapters and a fusion of adapters?

adapter_setup = [
                 [
        "rte",
        # TODO: Add your other adapter names here. 
        'mrpc',
        'mnli',
        'sick'
    ]
]
model.train_fusion(adapter_setup)

In [ ]:
# Check out your training args
print(training_args)

TrainingArguments(output_dir='./models/rte', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=64, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec02_17-07-14_8d8ba28de9c3', logging_first_step=False, logging_steps=50, save_steps=1000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=50, dataloader_num_workers=0, past_index=-1, run_name='./models/rte', disable_tqdm=False, remove_unused_columns=True

###### seed=2020

In [ ]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        do_save_full_model=False,
        do_save_adapter_fusion=True,
    )

trainer.train()
trainer.evaluate()

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with t

Step,Training Loss,Validation Loss,Acc
50,0.412307,0.811673,0.718412
100,0.210778,0.944624,0.736462
150,0.162032,1.012837,0.729242
200,0.154319,1.034165,0.722022


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 3.0, 'eval_acc': 0.7292418772563177, 'eval_loss': 1.0329092741012573}

###### seed=666

In [ ]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        do_save_full_model=False,
        do_save_adapter_fusion=True,
    )

trainer.train()
trainer.evaluate()

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


Step,Training Loss,Validation Loss,Acc
50,0.532900,0.562869,0.750903
100,0.398061,0.669980,0.736462
150,0.264239,0.833310,0.703971
200,0.203078,0.945233,0.718412


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 3.0, 'eval_acc': 0.703971119133574, 'eval_loss': 0.9585854411125183}

###### seed=1024

In [ ]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        do_save_full_model=False,
        do_save_adapter_fusion=True,
    )

trainer.train()
trainer.evaluate()

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/glue.py:77: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:521: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


Step,Training Loss,Validation Loss,Acc
50,0.546966,0.560257,0.736462
100,0.434188,0.619067,0.725632
150,0.326838,0.690678,0.725632
200,0.226932,0.835437,0.722022


/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'epoch': 3.0, 'eval_acc': 0.7220216606498195, 'eval_loss': 0.8418311476707458}